# Loop 14 Analysis: Strategic Assessment

## Current Status
- Best score: 70.659475 (multi-source ensemble)
- Target: 68.919154
- Gap: 1.740321 points (2.53%)

## Key Questions
1. What is the theoretical minimum score?
2. Where are the biggest opportunities for improvement?
3. What approaches haven't been tried?

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Polygon
from shapely import affinity
from shapely.ops import unary_union

# Load current best submission
df = pd.read_csv('/home/submission/submission.csv')
print(f"Total rows: {len(df)}")
print(df.head())

Total rows: 20100
        id          x          y       angle
0  001_000 -48.196086  58.770985   45.000000
1  002_000   0.154097  -0.038541  203.629378
2  002_001  -0.154097  -0.561459   23.629378
3  003_000   1.123656   0.781102  111.125132
4  003_001   1.234056   1.276000   66.370622


In [2]:
# Tree shape constants
TRUNK_W = 0.15
TRUNK_H = 0.2
BASE_W = 0.7
MID_W = 0.4
TOP_W = 0.25
TIP_Y = 0.8
TIER_1_Y = 0.5
TIER_2_Y = 0.25
BASE_Y = 0.0
TRUNK_BOTTOM_Y = -TRUNK_H

def get_tree_poly(x, y, deg):
    coords = [
        (0.0, TIP_Y), (TOP_W / 2.0, TIER_1_Y), (TOP_W / 4.0, TIER_1_Y),
        (MID_W / 2.0, TIER_2_Y), (MID_W / 4.0, TIER_2_Y), (BASE_W / 2.0, BASE_Y),
        (TRUNK_W / 2.0, BASE_Y), (TRUNK_W / 2.0, TRUNK_BOTTOM_Y),
        (-TRUNK_W / 2.0, TRUNK_BOTTOM_Y), (-TRUNK_W / 2.0, BASE_Y),
        (-BASE_W / 2.0, BASE_Y), (-MID_W / 4.0, TIER_2_Y), (-MID_W / 2.0, TIER_2_Y),
        (-TOP_W / 4.0, TIER_1_Y), (-TOP_W / 2.0, TIER_1_Y),
    ]
    poly = Polygon(coords)
    return affinity.translate(affinity.rotate(poly, deg, origin=(0, 0)), x, y)

# Calculate tree area
tree = get_tree_poly(0, 0, 0)
tree_area = tree.area
print(f"Tree area: {tree_area:.6f}")
print(f"Tree bounding box at 0 deg: {tree.bounds}")

# Calculate bounding box dimensions at different angles
for angle in [0, 45, 90, 135]:
    t = get_tree_poly(0, 0, angle)
    bounds = t.bounds
    width = bounds[2] - bounds[0]
    height = bounds[3] - bounds[1]
    print(f"Angle {angle}: width={width:.4f}, height={height:.4f}, bbox_area={width*height:.4f}")

Tree area: 0.245625
Tree bounding box at 0 deg: (-0.35, -0.2, 0.35, 0.8)
Angle 0: width=0.7000, height=1.0000, bbox_area=0.7000
Angle 45: width=0.8132, height=0.8132, bbox_area=0.6613
Angle 90: width=1.0000, height=0.7000, bbox_area=0.7000
Angle 135: width=0.8132, height=0.8132, bbox_area=0.6613


In [3]:
# Calculate score breakdown by N
def calculate_score_for_n(df, n):
    prefix = f"{n:03d}_"
    group = df[df['id'].str.startswith(prefix)].sort_values('id')
    if len(group) != n:
        return None
    
    xs = group['x'].values.astype(float)
    ys = group['y'].values.astype(float)
    degs = group['angle'].values.astype(float)
    
    min_x = min_y = 1e10
    max_x = max_y = -1e10
    for i in range(n):
        poly = get_tree_poly(xs[i], ys[i], degs[i])
        bounds = poly.bounds
        if bounds[0] < min_x: min_x = bounds[0]
        if bounds[1] < min_y: min_y = bounds[1]
        if bounds[2] > max_x: max_x = bounds[2]
        if bounds[3] > max_y: max_y = bounds[3]
    
    side = max(max_x - min_x, max_y - min_y)
    return side * side / n

# Calculate scores for all N
scores = {}
for n in range(1, 201):
    score = calculate_score_for_n(df, n)
    if score:
        scores[n] = score

print(f"Total score: {sum(scores.values()):.6f}")
print(f"\nTop 10 contributors to score:")
for n, s in sorted(scores.items(), key=lambda x: -x[1])[:10]:
    print(f"  N={n}: {s:.6f} ({s/sum(scores.values())*100:.2f}%)")

print(f"\nBottom 10 contributors to score:")
for n, s in sorted(scores.items(), key=lambda x: x[1])[:10]:
    print(f"  N={n}: {s:.6f} ({s/sum(scores.values())*100:.2f}%)")

Total score: 70.659475

Top 10 contributors to score:
  N=1: 0.661250 (0.94%)
  N=2: 0.450779 (0.64%)
  N=3: 0.434745 (0.62%)
  N=5: 0.416850 (0.59%)
  N=4: 0.416545 (0.59%)
  N=7: 0.399897 (0.57%)
  N=6: 0.399610 (0.57%)
  N=9: 0.387415 (0.55%)
  N=8: 0.385407 (0.55%)
  N=15: 0.379203 (0.54%)

Bottom 10 contributors to score:
  N=181: 0.329946 (0.47%)
  N=156: 0.329987 (0.47%)
  N=182: 0.329988 (0.47%)
  N=180: 0.331001 (0.47%)
  N=155: 0.332074 (0.47%)
  N=168: 0.332475 (0.47%)
  N=179: 0.332595 (0.47%)
  N=195: 0.332610 (0.47%)
  N=167: 0.332835 (0.47%)
  N=194: 0.332999 (0.47%)


In [4]:
# Theoretical minimum analysis
# For N trees, the minimum bounding box area is N * tree_area (if perfect packing)
# The score is side^2 / N, so minimum score is tree_area (if perfect square packing)

print("Theoretical minimum analysis:")
print(f"Tree area: {tree_area:.6f}")
print(f"If perfect packing (100% efficiency), score per N = tree_area = {tree_area:.6f}")
print(f"Theoretical minimum total score (200 N values): {tree_area * 200:.6f}")

# Calculate packing efficiency for each N
print("\nPacking efficiency by N:")
efficiencies = {}
for n in [1, 2, 5, 10, 20, 50, 100, 150, 200]:
    if n in scores:
        # score = side^2 / n, so side^2 = score * n
        # packing efficiency = (n * tree_area) / side^2 = (n * tree_area) / (score * n) = tree_area / score
        efficiency = tree_area / scores[n]
        efficiencies[n] = efficiency
        print(f"  N={n}: score={scores[n]:.6f}, efficiency={efficiency*100:.2f}%")

Theoretical minimum analysis:
Tree area: 0.245625
If perfect packing (100% efficiency), score per N = tree_area = 0.245625
Theoretical minimum total score (200 N values): 49.125000

Packing efficiency by N:
  N=1: score=0.661250, efficiency=37.15%
  N=2: score=0.450779, efficiency=54.49%
  N=5: score=0.416850, efficiency=58.92%
  N=10: score=0.376630, efficiency=65.22%
  N=20: score=0.376057, efficiency=65.32%
  N=50: score=0.360753, efficiency=68.09%
  N=100: score=0.345531, efficiency=71.09%
  N=150: score=0.337064, efficiency=72.87%
  N=200: score=0.337564, efficiency=72.76%


In [5]:
# Analyze the gap to target
target = 68.919154
current = sum(scores.values())
gap = current - target

print(f"Current score: {current:.6f}")
print(f"Target score: {target:.6f}")
print(f"Gap: {gap:.6f} ({gap/current*100:.2f}%)")

# How much improvement needed per N on average?
print(f"\nAverage improvement needed per N: {gap/200:.6f}")

# Which N values have the most room for improvement?
print("\nN values with worst packing efficiency (most room for improvement):")
for n, eff in sorted(efficiencies.items(), key=lambda x: x[1])[:10]:
    potential_improvement = scores[n] - tree_area
    print(f"  N={n}: efficiency={eff*100:.2f}%, potential improvement={potential_improvement:.6f}")

Current score: 70.659475
Target score: 68.919154
Gap: 1.740321 (2.46%)

Average improvement needed per N: 0.008702

N values with worst packing efficiency (most room for improvement):
  N=1: efficiency=37.15%, potential improvement=0.415625
  N=2: efficiency=54.49%, potential improvement=0.205154
  N=5: efficiency=58.92%, potential improvement=0.171225
  N=10: efficiency=65.22%, potential improvement=0.131005
  N=20: efficiency=65.32%, potential improvement=0.130432
  N=50: efficiency=68.09%, potential improvement=0.115128
  N=100: efficiency=71.09%, potential improvement=0.099906
  N=200: efficiency=72.76%, potential improvement=0.091939
  N=150: efficiency=72.87%, potential improvement=0.091439


In [6]:
# Analyze small N values in detail
print("Detailed analysis of small N values:")
for n in range(1, 11):
    if n in scores:
        # Get the configuration
        prefix = f"{n:03d}_"
        group = df[df['id'].str.startswith(prefix)].sort_values('id')
        xs = group['x'].values.astype(float)
        ys = group['y'].values.astype(float)
        degs = group['angle'].values.astype(float)
        
        # Calculate bounding box
        min_x = min_y = 1e10
        max_x = max_y = -1e10
        for i in range(n):
            poly = get_tree_poly(xs[i], ys[i], degs[i])
            bounds = poly.bounds
            if bounds[0] < min_x: min_x = bounds[0]
            if bounds[1] < min_y: min_y = bounds[1]
            if bounds[2] > max_x: max_x = bounds[2]
            if bounds[3] > max_y: max_y = bounds[3]
        
        width = max_x - min_x
        height = max_y - min_y
        side = max(width, height)
        
        print(f"\nN={n}:")
        print(f"  Score: {scores[n]:.6f}")
        print(f"  Bounding box: {width:.4f} x {height:.4f}")
        print(f"  Side length: {side:.4f}")
        print(f"  Angles: {degs}")
        print(f"  Efficiency: {tree_area/scores[n]*100:.2f}%")

Detailed analysis of small N values:

N=1:
  Score: 0.661250
  Bounding box: 0.8132 x 0.8132
  Side length: 0.8132
  Angles: [45.]
  Efficiency: 37.15%

N=2:
  Score: 0.450779
  Bounding box: 0.9495 x 0.9495
  Side length: 0.9495
  Angles: [203.629378  23.629378]
  Efficiency: 54.49%

N=3:
  Score: 0.434745
  Bounding box: 1.1420 x 1.1420
  Side length: 1.1420
  Angles: [111.12513229  66.37062227 155.13405194]
  Efficiency: 56.50%

N=4:
  Score: 0.416545
  Bounding box: 1.2908 x 1.2908
  Side length: 1.2908
  Angles: [156.37062215 156.37062227 336.37062227 336.37062215]
  Efficiency: 58.97%

N=5:
  Score: 0.416850
  Bounding box: 1.4437 x 1.4437
  Side length: 1.4437
  Angles: [  293.62937773    23.62937774   112.57350742    66.35879844
 12809.95523713]
  Efficiency: 58.92%

N=6:
  Score: 0.399610
  Bounding box: 1.5484 x 1.5484
  Side length: 1.5484
  Angles: [293.62937773 338.87232687 338.87232686  23.62937773 246.37062227
 158.87232686]
  Efficiency: 61.47%

N=7:
  Score: 0.399897
 

In [7]:
# Check N=1 specifically - what is the optimal angle?
print("N=1 Analysis:")
print("Testing different angles for N=1:")

best_score = float('inf')
best_angle = None

for angle in range(0, 360, 1):
    poly = get_tree_poly(0, 0, angle)
    bounds = poly.bounds
    width = bounds[2] - bounds[0]
    height = bounds[3] - bounds[1]
    side = max(width, height)
    score = side * side / 1
    
    if score < best_score:
        best_score = score
        best_angle = angle

print(f"Best angle for N=1: {best_angle} degrees")
print(f"Best score for N=1: {best_score:.6f}")
print(f"Current N=1 score: {scores[1]:.6f}")
print(f"Potential improvement: {scores[1] - best_score:.6f}")

# More precise search around best angle
print("\nFine-grained search around best angle:")
for angle in np.arange(best_angle - 5, best_angle + 5, 0.1):
    poly = get_tree_poly(0, 0, angle)
    bounds = poly.bounds
    width = bounds[2] - bounds[0]
    height = bounds[3] - bounds[1]
    side = max(width, height)
    score = side * side / 1
    
    if score < best_score:
        best_score = score
        best_angle = angle

print(f"Best angle for N=1 (fine): {best_angle:.1f} degrees")
print(f"Best score for N=1 (fine): {best_score:.6f}")

N=1 Analysis:
Testing different angles for N=1:
Best angle for N=1: 45 degrees
Best score for N=1: 0.661250
Current N=1 score: 0.661250
Potential improvement: -0.000000

Fine-grained search around best angle:
Best angle for N=1 (fine): 45.0 degrees
Best score for N=1 (fine): 0.661250


In [ ]:
# Summary of findings
print("="*60)
print("SUMMARY OF FINDINGS")
print("="*60)
print(f"\n1. Current best score: {current:.6f}")
print(f"2. Target score: {target:.6f}")
print(f"3. Gap: {gap:.6f} ({gap/current*100:.2f}%)")
print(f"\n4. Tree area: {tree_area:.6f}")
print(f"5. Theoretical minimum (perfect packing): {tree_area * 200:.6f}")
print(f"6. Current efficiency: {(tree_area * 200) / current * 100:.2f}%")
print(f"7. Target efficiency: {(tree_area * 200) / target * 100:.2f}%")

print(f"\n8. Small N values (1-10) contribute: {sum(scores[n] for n in range(1,11)):.6f} ({sum(scores[n] for n in range(1,11))/current*100:.2f}%)")
print(f"9. Large N values (100-200) contribute: {sum(scores[n] for n in range(100,201)):.6f} ({sum(scores[n] for n in range(100,201))/current*100:.2f}%)")

print("\n" + "="*60)
print("KEY INSIGHT: The gap to target (1.74 points) is 2.53% of current score.")
print("This requires either:")
print("  a) Finding 2.53% improvement across all N values")
print("  b) Finding large improvements in specific N values")
print("  c) Using techniques not available in public kernels")
print("="*60)